# Create clonemaps for PCR-GLOBWB
### PCR-GLOBWB uses binary maps (clone maps) to specify extent and resolution

### Snellius Dutch Cluster Parameter location: /gpfs/work1/0/dfguu/users/edwin/data/pcrglobwb_input_arise/develop/

In [ ]:
import os, math
import subprocess
import rioxarray
import xarray as xr
import geopandas as gpd
import pandas as pd

from glob import glob
from pathlib import Path

# Set Paths

In [ ]:
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/')
AUXDATA = Path(f"{ROOT}/aux_data")
MODELS = Path(f'{ROOT}/pcr-globwb/')
FORCING = Path(f'{ROOT}/forcing/')

# Config

In [ ]:
# Get available basin IDs wflow_sbm
basin_dirs = glob(f'{MODELS}/*')
basin_ids = [s.split('/')[-1] for s in basin_dirs]
basin_ids.remove('uk')
basin_ids.sort()

# Read boundary shapefiles

In [ ]:
# Read shapefile
gdf = gpd.read_file(f"{AUXDATA}/CAMELS-GB/data/CAMELS_GB_catchment_boundaries/CAMELS_GB_catchment_boundaries.shp")
gdf = gdf.set_index('ID_STRING')

# Reproject shapefile
gdf = gdf.to_crs({'init':'epsg:4326'})

In [ ]:
# Get maximum extent
minx = gdf.bounds.minx.min()
miny = gdf.bounds.miny.min()

maxx = gdf.bounds.maxx.max()
maxy = gdf.bounds.maxy.max()

print(minx,miny,maxx,maxy)

# Create clonemaps function

In [ ]:
def create_clonemap(basin_id, lonmin,latmin,lonmax,latmax):
    global_clonemap = f"{AUXDATA}/PCR-GLOBWB/global_30sec_clone.map"
    output_clonemap = f"{MODELS}/{basin_id}/{basin_id}_30sec_clone.map"
    
    # Remove existing file
    if output_clonemap:
        OUTPUT = Path(output_clonemap)
        OUTPUT.unlink(output_clonemap)

    subprocess.call(
        f"gdal_translate -of PCRaster {global_clonemap} -projwin "
        f"{lonmin} {latmax} {lonmax} {latmin} {output_clonemap}",
        shell=True,
    )

    return print(f'finished: {basin_id}')

In [ ]:
def create_nc_clone(basin_id, lonmin,latmin,lonmax,latmax):
    global_nc_clone = f"{AUXDATA}/PCR-GLOBWB/global_30sec_clone_correct_lat.nc"
    output_nc_clone = f"{MODELS}/{basin_id}/{basin_id}_30sec_nc_clone.nc"
    
    # Remove existing file
    if output_nc_clone:
        OUTPUT = Path(output_nc_clone)
        OUTPUT.unlink(output_nc_clone)
        
    subprocess.call(
        f"gdal_translate -of NetCDF {global_nc_clone} -projwin "
        f"{lonmin} {latmax} {lonmax} {latmin} {output_nc_clone}",
        shell=True,
    )
    
    return print(f'finished: {basin_id}')

## UK Clonemap

In [ ]:
basin_id = 'uk'

lonmin = -8.0
latmin = 50.0
lonmax = 1.5
latmax = 59.5

create_clonemap(basin_id, lonmin,latmin,lonmax,latmax)
create_nc_clone(basin_id, lonmin,latmin,lonmax,latmax)